<a href="https://colab.research.google.com/github/mafux777/kintusgi-extras/blob/kintsugi-x/Kintsugi_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's start with some utilities for substrate for python..

In [1]:
# next line only needed in Google Colaboratory! Otherwise pip install yourself
!pip install substrate-interface

from substrateinterface import SubstrateInterface

kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain"
)
ksm_substrate = SubstrateInterface(
    url="wss://kusama-rpc.dwellir.com"
)

def convert_kint_to_ksm(kint):
    universal = kint_substrate.ss58_decode(kint)
    kusama = ksm_substrate.ss58_encode(universal)
    return kusama

def convert_ksm_to_kint(k):
    universal = ksm_substrate.ss58_decode(k)
    kint = kint_substrate.ss58_encode(universal)
    return kint



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 5.2 MB/s 
     |████████████████████████████████| 1.0 MB 40.4 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 1.0 MB 36.0 MB/s 
     |████████████████████████████████| 856 kB 28.8 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.8 MB/s 
     |████████████████████████████████| 243 kB 46.9 MB/s 
     |████████████████████████████████| 2.0 MB 38.1 MB/s 
     |████████████████████████████████| 119 kB 50.0 MB/s 
     |████████████████████████████████| 481 kB 17.0 MB/s 
     |████████████████████████████████| 62 kB 629 kB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236741 sha256=1c51166c6a47cc2ab7ba1ae9bf55200374ad7040934f32187296db5463011a64
  Stored in directory: /root/.cache/pip/wheels

Let's analyze Kintsugi's vaults and early investors. For that, we will use several subsquids.

Kintsugi-X is a subsquid built by the Kintsugi-X team for the purpose of analysing transfers.

In [2]:
import requests
import json
import pandas as pd

kintsugi = "https://api-kusama.interlay.io/graphql/graphql"
kusama = "https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql"
kintsugi_x = "https://app.gc.subsquid.io/beta/kintsugi-x/v3/graphql"


One of the interesting questions is: do vaults self-mint?

In [3]:
def is_equal(row):
    return row["userParachainAddress"] == row['vault.accountId']


Let's start creating a directory of interesting Kintsugi addresses.
This info comes from the Interlay discord server, "meet the vaults".

In [4]:
kts = {
    "a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs": ['Simon Kraus', '@sik'],
    "a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63": ['Simon Kraus', '@sik'],
    "a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo": ['Simon Kraus', '@sik'],
    "a3bccyaV6tCoqaWqByeqvXo5kBbd1m4yCPKyX937HB1APTt7Y": ['rodrigo.barrios', 'hypersphere'],
    "a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs": ['@boyswan'],
    "a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6": ['@seergeist'],
    "a3dh62XsvNmtPAzfwCH9bv34dqPFzjKcWBi5mYM93mKmMt64s": ['@DkA7s'],
    "a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK": ['@whisperit'],
    "a3aDPraojQvYhVHjyVuYRFXno58EPMjegrY9nubPmpck2X7JS": ['@blinkin', 'chaos DAO'],
    "a3azGTG3qGmUuQckCKjFAhfjfnnRAXmpgV4fPVPziNaA1zCwG": ['@marvel'],
    "a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS": ['@mafux777'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3baaLbC1JMHJLJ2HwEQMz3S5VuiCWBYy4i66Ziq1vXzmVU6b": ['@spazcoin', 'chaos DAO'],
    "a3fudELrRCjuSyYEPkRAKFQyjzo5YyU228LdqinGsnjBUNB8P": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO'],
    "a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed": ['@spazcoin', 'chaos DAO'],
    "a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4": ['@timbotronic'],
    "a3fZSzXxTZYY58BQrfhJx8cDtp4wRdbZ8X4ReF2iUT63y5RcX": ['@0xvault'],
    "a3dJfVzssBJgBmRuMZBre5H71rvawHJoFvFHGz2Aq7Hdt492w": ['@niko'],
    "a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti": ['@paride'],
    "a3cAyFZMgahPoAyWbNRrjX2TnXQtpS3bztCVMNuLNcTYATBte": ['@dan', 'interlay'],
    "a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe": ['pumpernickel'],
    "a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE": ['@alibaba'],
    "a3cDUVdQi8FqkiUBMjqS3RBWPghRWFXBwjRzKZvmB3MmHLDcP": ['@alibaba'],
    "a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1": ['@warinelly'], # info provided by @quin
}

In [5]:
def add_label_to_list(my_list, label):
  for a in my_list:
    if a not in kts:
      kts[a] = []
    if label not in kts[a]:
      kts[a].append(label)    
  
def enrich_df(df, col):
  df[f'label_{col}'] = df[col].apply(lambda v: "/".join(kts.get(v, [])))

Let's create a query to understand transfers from Kusama (Relay Chain) to Kintsugi. These have to be KSM transfers, because Kusama itself does not handle other assets.
We assume that all of these will end up as collateral. 


In [6]:
to_kintsugi = """
query MyQuery {
  transfers(where: {name_eq: "xcmPallet.reserveTransferAssets", AND: {to: {id_eq: "F7fq1inhrJsYSUkWhyZ3zqtp5K3AKBBjbPWy6VLiRGHipPi"}}}, orderBy: date_DESC) {
    name
    amount
    from {
      id
    }
    date
    to {
      id
    }
  }
}

"""

In [7]:
r = requests.post("https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql", json={"query" : to_kintsugi}).json()
df_0 = pd.json_normalize(r['data']['transfers'])
df_0['ksm'] = df_0['amount'].apply(lambda x: float(x) / 1e12)
kusama_transfers = df_0.groupby('from.id').agg(dict(ksm="sum", date="min")).sort_values('ksm', ascending=False)
kusama_transfers.reset_index(inplace=True)
kusama_transfers.rename(columns={"from.id": "kusama",
                                 "date": "first_seen"}, inplace=True)
kusama_transfers['kintsugi'] = kusama_transfers.kusama.apply(convert_ksm_to_kint)
kusama_transfers


,kusama,ksm,first_seen,kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2940.17498,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,1067.96346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,859.13600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW
...,...,...,...,...
59,J1dYYtLHd4XwdeBrZVELtXxBoabUnXJbfc8oiNvNACpdXqT,0.10000,2022-05-22T02:00:54.028Z,a3feTzhfmfSe3d1RhL1PQe8KGy247afquSZVpy4Bb6pMsphfZ
60,HcNCnh9ZLYRcXB7wm3aPyhzS4i3ghoKP6UR4intuJvp8buf,0.05000,2022-04-25T18:25:06.087Z,a3fFCewUawA7wHtxdRCwkhDVKDHBZnb7vDzN7E4bZdy5sKwfN
61,D8yhHcbwNuYEJUoSDwyQXVnGJzfGpEpnRq8Qdmkc1Etw8vE,0.05000,2022-04-25T19:23:30.004Z,a3amp9SQ3KCV3ugGJufr9hmH73XUBNhZRdKipZyaRLfPx8dqr
62,DoSsDYsK1a8vMaUC3ZhVzvnpKfPZqbBXXQ5cDssxSPgtNqJ,0.01000,2022-05-24T06:51:00.033Z,a3bSHKNLJgq9ebjMyfVTsoEi7bY8ufiunNRHmmZgYh6Yk5gnz


Let's identify all the accounts who crowdloaned to Kintsugi!

In [8]:
crowdloans = """
query CrowloanersKintsugi {
  crowdloans(where: {para: {id_eq: "2092"}}) {
    contributions {
      amount
      timestamp
      account {
        id
        substrateAccount
      }
    }
  }
}
"""
r = requests.post("https://app.gc.subsquid.io/beta/parity-kusama/1/graphql", json={"query" : crowdloans}).json()
crd = pd.json_normalize(r['data']['crowdloans'][0]['contributions'])
crd['ksm'] = crd['amount'].apply(lambda x: float(x) / 1e12)
crowdloans = crd.groupby(['account.id', 'account.substrateAccount']).agg(
    dict(
        ksm=sum,
        timestamp=min
    )
)
crowdloans.describe()

,ksm
count,8013.000000
mean,24.959432
std,486.510228
min,0.100000
25%,0.111000
50%,1.000000
75%,2.100000
max,22688.000000


In [9]:
q = crowdloans.ksm.quantile(.8)
top_loaners = crowdloans[crowdloans.ksm>q]
top_loaners.reset_index(inplace=True)
top_loaners.rename(columns={"account.id":"kusama",
                            "timestamp": "first_seen"}, inplace=True)
top_loaners['kintsugi'] = top_loaners.kusama.apply(convert_ksm_to_kint)

top_loaners.loc[:, ['kusama', 'kintsugi', 'ksm', 'first_seen']].sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,kusama,kintsugi,ksm,first_seen
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,22688.00000,2021-09-24T10:52:42.022Z
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,18517.00000,2021-09-24T10:53:18.003Z
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,17270.44313,2021-09-21T23:42:36.005Z
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,15001.00000,2021-09-17T06:32:06.005Z
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,10000.00000,2021-09-16T11:15:06.007Z
...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,3.05000,2021-09-25T18:32:24.006Z
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,3.05000,2021-09-25T10:54:48.008Z
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,3.04570,2021-09-24T19:57:06.012Z
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,3.04000,2021-09-24T22:29:36.014Z


In [10]:
redeem_query = """query MyQuery {
  redeems(orderBy: request_timestamp_ASC) {
    id
    request {
      requestedAmountBacking
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    userBackingAddress
    bridgeFee
    btcTransferFee
    collateralPremium
    status
    execution {
      height {
        absolute
        active
      }
      timestamp
    }
    cancellation {
      timestamp
      slashedCollateral
      reimbursed
      height {
        absolute
        active
      }
    }
  }
}
"""


Let's use the official Kintsugi squid to download data about issue requests.

In [11]:
issue_query = """
query MyQuery {
  issues(orderBy: request_timestamp_DESC, limit: 10000, offset: 0) {
    id
    request {
      amountWrapped
      bridgeFeeWrapped
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    vaultBackingAddress
    vaultWalletPubkey
    griefingCollateral
    status
    refund {
      amountPaid
      btcAddress
      btcFee
      executionHeight {
        absolute
        active
      }
      executionTimestamp
      id
      requestHeight {
        absolute
        active
      }
      requestTimestamp
    }
    execution {
      height {
        absolute
        active
      }
      amountWrapped
      bridgeFeeWrapped
      timestamp
    }
    cancellation {
      timestamp
      height {
        absolute
        active
      }
    }
  },
}
"""

In [12]:
# Obtain a list of all redemptions and summarize them a bit
r = requests.post(kintsugi, json={"query" : redeem_query}).json()
df_1 = pd.json_normalize(r['data']['redeems'])
df_1['btc'] = df_1['request.requestedAmountBacking'].apply(lambda x: float(x) / -1e8)
df_1['self'] = df_1.apply(is_equal, axis=1)
df_1['action'] = "redeem"

# Obtain a list of all issue executions and summarize them a bit
# issue_query = get_query_text_from_file("issue")
r = requests.post(kintsugi, json={"query" : issue_query}).json()
df_2 = pd.json_normalize(r['data']['issues'])
df_2['btc'] = df_2['request.amountWrapped'].apply(lambda x: float(x) / 1e8)
df_2['self'] = df_2.apply(is_equal, axis=1)
df_2['action'] = "issue"

cols_1 = set(df_1.columns) - set(df_2.columns)
cols_2 = set(df_2.columns) - set(df_1.columns)


Let's download data about redemptions, so we can net out issues and redemptions for calculating the vault sizes.
TODO: This analysis should also include Theft and Replacement.
The 80% quantile gives us the accounts responsible for 80% of the issues & redemptions

In [13]:
redeems = df_1.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=True)
redeems['btc'] = redeems.btc.apply(lambda x: -x)
q = redeems.btc.quantile(.8)
top_redeemers = redeems[redeems.btc>q].index
add_label_to_list(top_redeemers, 'Top Redeemer')
redeems['btc'] = redeems.btc.apply(lambda x: -x)


In [14]:
issues = df_2.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=False)
q = issues.btc.quantile(.8)
top_issues = issues[issues.btc>q].index
add_label_to_list(top_issues, 'Top Issuer')



In [15]:
issues[issues.btc>q].btc.sum()

52.130211499999994

In [16]:
self_issuers = df_2[df_2.self==True]
self_issuers.groupby("userParachainAddress").agg(dict(btc=sum)).sort_values("btc", ascending=False)

,btc
userParachainAddress,
a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,7.358758
a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,3.518315
a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.558984
a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650
a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.787367
a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK,0.757999
a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,0.734373
a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed,0.619498
a3dCFNgcQsgL4wX6apeaxVLTyRzeCiUdEczyvPcXZ5Dmg5rez,0.601403


Let's consolidate this info and merge the two dataframes so we can net out the issue requests and the redeem requests.

In [17]:
merged_df = pd.concat([
    df_1.loc[df_1.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
    df_2.loc[df_2.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
                       ]).sort_values('request.timestamp')
# apportion BTC to self or other
merged_df['btc_self'] = merged_df.apply(lambda b: b.btc if b.self else 0, axis=1)
merged_df['btc_other'] = merged_df.apply(lambda b: b.btc if not b.self else 0, axis=1)
# add up the BTC depending on self/other
biggest_vaults = merged_df.groupby('vault.accountId').agg(dict(btc_self='sum', btc_other='sum', btc='sum')).sort_values('btc', ascending=False)
biggest_vaults['social'] = biggest_vaults.apply(lambda b: True if b.btc_self<0.2*b.btc else False, axis=1)
biggest_vaults['selfish'] = biggest_vaults.apply(lambda b: True if b.btc_self>0.8*b.btc else False, axis=1)
add_label_to_list(biggest_vaults.loc[biggest_vaults.social].index, 'Social')
add_label_to_list(biggest_vaults.loc[biggest_vaults.selfish].index, 'Selfish')


biggest_vaults.reset_index(inplace=True)
enrich_df(biggest_vaults, "vault.accountId")
total_btc = biggest_vaults.btc.sum()
biggest_vaults['share'] = biggest_vaults.btc.apply(lambda btc: f"{btc/total_btc:.1%}")
biggest_vaults


,vault.accountId,btc_self,btc_other,btc,social,selfish,label_vault.accountId,share
0,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650,12.268932,13.167582,True,False,Top Issuer/Social,45.4%
1,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,3.982269,0.114068,4.096337,False,True,Top Redeemer/Top Issuer/Selfish,14.1%
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,-0.098253,1.398658,1.300405,True,False,@warinelly/Social,4.5%
3,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,0.744558,0.183030,0.927588,False,True,Top Redeemer/Top Issuer/Selfish,3.2%
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,-1.738186,2.610267,0.872081,True,False,Top Redeemer/Social,3.0%
5,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.787357,-0.004999,0.782358,False,True,@mafux777/Top Issuer/Selfish,2.7%
6,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.000000,0.625398,0.625398,True,False,@paride/Social,2.2%
7,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,-0.021208,0.621429,0.600221,True,False,Social,2.1%
8,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.114897,-0.546023,0.568873,False,True,Top Redeemer/Top Issuer/Selfish,2.0%
9,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,0.000000,0.544708,0.544708,True,False,Social,1.9%


In [18]:
vaults = biggest_vaults.reset_index()
del vaults['index']
vaults = vaults.rename(columns={"vault.accountId":"vault"})
add_label_to_list(list(vaults.vault.iloc[0:20]), "Top 20 Vault")
add_label_to_list(list(vaults.vault.iloc[20:]), "Vault")
add_label_to_list(list(vaults.vault.loc[vaults.vault.isin(self_issuers.userParachainAddress)].iloc[0:20]), "Self Issuer")
enrich_df(vaults, 'vault')
vaults.iloc[0:25]

,vault,btc_self,btc_other,btc,social,selfish,label_vault.accountId,share,label_vault
0,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650,12.268932,13.167582,True,False,Top Issuer/Social,45.4%,Top Issuer/Social/Top 20 Vault/Self Issuer
1,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,3.982269,0.114068,4.096337,False,True,Top Redeemer/Top Issuer/Selfish,14.1%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,-0.098253,1.398658,1.300405,True,False,@warinelly/Social,4.5%,@warinelly/Social/Top 20 Vault
3,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,0.744558,0.183030,0.927588,False,True,Top Redeemer/Top Issuer/Selfish,3.2%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,-1.738186,2.610267,0.872081,True,False,Top Redeemer/Social,3.0%,Top Redeemer/Social/Top 20 Vault/Self Issuer
5,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.787357,-0.004999,0.782358,False,True,@mafux777/Top Issuer/Selfish,2.7%,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...
6,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.000000,0.625398,0.625398,True,False,@paride/Social,2.2%,@paride/Social/Top 20 Vault
7,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,-0.021208,0.621429,0.600221,True,False,Social,2.1%,Social/Top 20 Vault
8,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.114897,-0.546023,0.568873,False,True,Top Redeemer/Top Issuer/Selfish,2.0%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
9,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,0.000000,0.544708,0.544708,True,False,Social,1.9%,Social/Top 20 Vault


In [19]:
kintsugix_query="""
query MyQuery {
  transfers(orderBy: timestamp_DESC) {
    amount
    from {
      karura
      kintsugi
      kusama
      moonriver
      id
    }
    fromChain
    timestamp
    to {
      id
      karura
      kintsugi
      kusama
      moonriver
    }
    toChain
    token
  }
}

"""

In [20]:
r = requests.post(kintsugi_x, json={"query" : kintsugix_query}).json()
xtoken_transfers = pd.json_normalize(r['data']['transfers'])


In [21]:
master_t = dict(
    KSM = 1e12,
    BTC = 1e8,
    KBTC = 1e8,
    KINT = 1e12,
)

def fix_currency(row):
    token = row['token']
    divisor = master_t.get(token, 1.0)
    row[token.lower()] = float(row.get('amount', 0.0)) / divisor
    return row



Where did the biggest vaults send their tokens to or receive tokens from?

Let's see who the vault owners got the KINT from to start

In [22]:
xtoken_transfers.rename(columns={"from.id":"from_id", "to.id":"to_id"}, inplace=True)
xtoken_transfers = xtoken_transfers.apply(fix_currency, axis=1)
xtoken_transfers.columns

Index(['amount', 'dot', 'from.karura', 'from.kintsugi', 'from.kusama',
       'from.moonriver', 'fromChain', 'from_id', 'kbtc', 'kint', 'ksm',
       'timestamp', 'to.karura', 'to.kintsugi', 'to.kusama', 'to.moonriver',
       'toChain', 'to_id', 'token'],
      dtype='object')

In [23]:
xtoken_transfers.timestamp.max()

'2022-06-17T09:12:12.728000Z'

There are a few accounts involved in crowdloans and other "shenanigans". Let's label them, too. 

In [24]:
# how many top loaners received some KINT?
top_loaners.kintsugi
xtoken_transfers.loc[xtoken_transfers.to_id.isin(top_loaners.kintsugi)].groupby('from_id').agg({"kint":sum, "timestamp": min,}).sort_values('timestamp', ascending=True)

,kint,timestamp
from_id,,
a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,45.030000,2022-02-18T11:25:00.271000Z
a3azovNFWpdqTMKpqdasdhhx1HFVaTviiwkysahuPbiaT6uKu,0.947711,2022-02-20T16:27:18.619000Z
a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,6785.847347,2022-02-21T06:26:30.255000Z
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,24139.621779,2022-02-23T07:59:42.268000Z
a3d7Pp3bYwqF5CG5x2VamKXxrL89KXq3uHfkT7U9qTWP7WgBS,25.328090,2022-02-23T13:10:48.334000Z
...,...,...
a3cwAfHVeuvqSsZzXoY9MqmbmqgVvQ1sMN2txMEsf8MunaLJ3,0.050000,2022-06-10T16:36:00.132000Z
a3aoTwKsLJvPJwGNW4M7qguZECD4qUhXTowPKXABFG4sDga9S,51.100000,2022-06-13T18:48:06.324000Z
a3cgeH7CzXoGgXh453eaSJRCvbbBKZN4mejwUVkic8efQUi5R,536.034928,2022-06-13T20:05:18.479000Z


In [25]:
shenanigans = xtoken_transfers.loc[xtoken_transfers.toChain==2092].groupby('from_id').agg(dict(kint=sum, to_id='nunique'))
shenanigans.sort_values('to_id', ascending=False).iloc[0:20]

,kint,to_id
from_id,,
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,153800.622828,554
a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,7634.632533,441
a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,59829.943280,171
a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,6716.357000,100
a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,46886.048457,82
a3cgeH7CzXoGgXh453eaSJRCvbbBKZN4mejwUVkic8efQUi5R,942.619483,71
a3cgeH7D28bBsH77KFYdoMgyiXUHdk98XT5M2Wv5EgC45Kqya,4421.256144,29
a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,82.154192,18
a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,534.494270,13


That analysis seems to suggest we should label the ones which sent more than 5 KINT as "shenanigans"...


In [26]:
shenanigans_list = shenanigans.loc[shenanigans.to_id>5].index
shenanigans.reset_index(inplace=True)
enrich_df(shenanigans, 'from_id')

add_label_to_list(shenanigans_list, "shenanigans")
enrich_df(shenanigans, 'from_id')
shenanigans.sort_values('to_id', ascending=False).iloc[0:20]


,from_id,kint,to_id,label_from_id
41,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,153800.622828,554,shenanigans
573,a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,7634.632533,441,shenanigans
242,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,59829.943280,171,shenanigans
409,a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,6716.357000,100,shenanigans
338,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,46886.048457,82,shenanigans
374,a3cgeH7CzXoGgXh453eaSJRCvbbBKZN4mejwUVkic8efQUi5R,942.619483,71,shenanigans
375,a3cgeH7D28bBsH77KFYdoMgyiXUHdk98XT5M2Wv5EgC45Kqya,4421.256144,29,shenanigans
578,a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,82.154192,18,shenanigans
262,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,534.494270,13,@seergeist/Top Redeemer/Top Issuer/Selfish/Vau...
25,a3aPp8Ei4Qm3dTr57rRabVJhFKpGdzdxYkgFFgqVNZ7nqudyw,3.100000,12,shenanigans


Who are the accounts that have sent KINT or KSM to the top 20 vaults? We shall exclude "shenanigans".

to_id: the top 20 vaults

from_id: the accounts sending KINT excluding shenanigans



In [27]:
from tensorflow.python.profiler import trace
funding_accounts = xtoken_transfers.loc[
                                        (xtoken_transfers.to_id.isin(vaults.iloc[0:55, 0])) & 
                                        ~(xtoken_transfers.from_id.isin(shenanigans_list)) & 
                                        (xtoken_transfers.toChain==2092)
                                        ].groupby(['to_id', 'from_id', 'from.kusama','fromChain']).agg(dict(kint=sum, ksm=sum, timestamp=min))
funding_accounts.reset_index(inplace=True)
funding_accounts.rename(columns=dict(to_id="vault", from_id="daddy"), inplace=True)
funding_accounts.sort_values('timestamp')


,vault,daddy,from.kusama,fromChain,kint,ksm,timestamp
50,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,a3c58xGJ7YnPVHCLbwnksPsLGVFzLL9rdYFwdLMTD3tCRoeQ7,ESJW7WgAxoybpZ6ULrh6dYwi3WpEGY2hN3wB1eYKE3NcTHo,2092,2.000000,0.000000e+00,2022-03-23T13:01:24.625000Z
18,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,Ffj4XHRejwspbtpcWZHCrTpGVLSzsvynRPdS4N5Z8eajsrK,2092,11.000000,0.000000e+00,2022-03-23T17:19:54.491000Z
36,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,2092,15.000000,0.000000e+00,2022-03-23T17:20:12.525000Z
44,a3fRJZLKJEsNY2fcDDLtzh7MWAE5DbVykyEfBbKdPbJSGLQjh,a3dhLM47mvpftx2vtdM7fk8uZR4uen128xMgdxAewb1ueRuCF,G4VtuLLZ16PGf9P9uDVSu8EdrS8g7hY7TnwnprGrMkbEYre,2092,18.250000,1.201500e+02,2022-03-24T17:12:42.301000Z
46,a3fjAcNYScCN3sLnGsZZTfQVPUXT49o7FZL3iXLW7DVoyVbtL,a3fyW7jHWzXxrXsWnEiLab4vWBUphMN4hT73M1ZEzM2pVJNx9,JLffaW5ciPLrVjGmGSQHq9BQGMBFUk6cD9erDSKcNfS6qUE,2092,82.801000,3.048072e+00,2022-03-25T11:03:00.362000Z
42,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,a3adPv7McJ11sBVApm9ZcR52wpQAKBLFjd2n9E6DH1cBkccMg,CzZTxaAvBSMW7PKHhfS7qFd3Bgo5Sw8n8tT4kQcGx2hRCe1,2092,10.000000,0.000000e+00,2022-03-27T16:55:12.407000Z
25,a3chQwNk9sxP4WHnm82cD6a5AR2tXJ7bmJ88yZGQp4Ay957PB,a3adPv7McJ11sBVApm9ZcR52wpQAKBLFjd2n9E6DH1cBkccMg,CzZTxaAvBSMW7PKHhfS7qFd3Bgo5Sw8n8tT4kQcGx2hRCe1,2092,10.000000,0.000000e+00,2022-03-27T16:56:06.331000Z
41,a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,JKbZuQCY1Z7H5HJ1fXDobKbt29v8sFyyLXQuqdQ7axQtzdH,2092,210.000000,1.106600e+02,2022-03-28T10:07:36.474000Z
11,a3bAsW2TQb1c7HrcZd2y81SdCVSGEA5pRkfg576G2EyxQupbw,a3awoQLGoQNGvR7UuXvgA7WhEk954SYhLuxvXASenQ8z1LUQH,DJxxBVN2YhQjjhQ4UmypGuuxvbYLfNk552q16r7fUpx96Pr,2092,0.500000,0.000000e+00,2022-03-28T11:06:06.643000Z
5,a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,JKbZuQCY1Z7H5HJ1fXDobKbt29v8sFyyLXQuqdQ7axQtzdH,2092,14.900000,9.194816e+01,2022-03-28T12:46:12.326000Z


Let's label each daddy with his vault

In [28]:
for n, row in funding_accounts.iterrows():
  vault = row['vault'][-5:] # use only last 5 letters
  daddy = row['daddy']
  add_label_to_list([daddy], f'Daddy of {vault}')  
  add_label_to_list([row['vault']], f'{vault}')

In [29]:
add_label_to_list(funding_accounts.daddy, "Likely Vault Owner")
enrich_df(funding_accounts, 'vault')
enrich_df(funding_accounts, 'daddy')
funding_accounts.sort_values('ksm', ascending=False).loc[:, ['label_vault', 'label_daddy', 'kint', 'ksm']]


,label_vault,label_daddy,kint,ksm
23,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.000000,1.747100e+02
44,Social/Vault/Self Issuer/GLQjh,Daddy of GLQjh/Likely Vault Owner,18.250000,1.201500e+02
41,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJj...,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...,210.000000,1.106600e+02
5,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,Simon Kraus/@sik/Top Redeemer/Selfish/Vault/Da...,14.900000,9.194816e+01
21,Social/Top 20 Vault/Daddy of Poxg8/rEszH/Likel...,Social/Top 20 Vault/Poxg8/Daddy of rEszH/Daddy...,6.803410,5.033559e+01
22,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.000000,3.000000e+01
39,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJj...,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,855.000000,2.900000e+01
20,@seergeist/Top Redeemer/Top Issuer/Selfish/Vau...,Daddy of KueH6/Likely Vault Owner,2.069000,2.220283e+01
30,Social/Vault/3yHRD,Daddy of 3yHRD/Likely Vault Owner,1.000000,1.299987e+01
26,Selfish/Vault/Self Issuer/957PB,Simon Kraus/@sik/Selfish/Top 20 Vault/Self Iss...,0.000000,1.179000e+01


Of the funders, which ones do we know from the KSM analysis?

In [30]:
k = kusama_transfers.loc[kusama_transfers.ksm>50]
add_label_to_list(k.kintsugi, 'K>50')
enrich_df(k, 'kintsugi')
k

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,ksm,first_seen,kintsugi,label_kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2940.17498,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR,Vault/Self Issuer/BdzVR/K>50
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,1067.96346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,@paride/Social/Top 20 Vault/K>50
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,859.13600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...
5,Cdcb6dodh71D8rCf3sWsokuXGkaS1YKcA7mgGWbDuSrHSFA,814.00000,2022-05-16T14:33:36.024Z,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,@warinelly/Social/Top 20 Vault/uUvS1/K>50
6,EcdZSvpYwbhESMviUwYQb9WiZTgnTYrAsRLUf3GmXyzy5dH,811.65000,2022-04-25T10:08:12.007Z,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...
7,DMZC2aCdR5SjYTr9duiHFHWHSuqwfFUjHX1Qru9YLSe9H5e,666.48400,2022-05-01T20:50:54.012Z,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
8,GCE5wFsJX3x3MpPrekqnXe2STTye3gdvxvjX5gSXL5YSZ74,577.50000,2022-05-20T19:26:54.016Z,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,Social/Top 20 Vault/K>50
9,HyWvJx6EUsRV7zwtt3m5sfok8Z56cnf2oymLgu9johaLdRh,476.05000,2022-05-23T06:35:30.013Z,a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed,@spazcoin/chaos DAO/Top Issuer/Selfish/Top 20 ...


In [31]:
add_label_to_list(top_loaners.kintsugi, 'Top Crowdloaner')
enrich_df(top_loaners, 'kintsugi')
top_loaners.sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,account.substrateAccount,ksm,first_seen,kintsugi,label_kintsugi
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,5ERZ3Ec3V6H5jUrWr14vfL1ykKa2k1znFHPyKaWyj5u6mCfs,22688.00000,2021-09-24T10:52:42.022Z,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,Top Crowdloaner
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,5HKC1c1BtaSJv4MDSS5FVVSUue8egHU296UPFn1fTrbQtTYd,18517.00000,2021-09-24T10:53:18.003Z,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,Top Crowdloaner
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,5EF9yLw9dF5bvjrnAevvk61sV8jsPXdXhQyNvwr34zyqHC4W,17270.44313,2021-09-21T23:42:36.005Z,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,Top Crowdloaner
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,5DyYhD12vugbs5a4pLVdd9D5h57bDaTyTVdozFiYbj9uXow7,15001.00000,2021-09-17T06:32:06.005Z,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,Daddy of ceZ9g/Likely Vault Owner/Top Crowdloaner
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,5GmTYN43P49kehsNXzHm9RGT19dGGkwHDzcpasj7UFKjVW2J,10000.00000,2021-09-16T11:15:06.007Z,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,Top Crowdloaner
...,...,...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,5CofsbgRRS8Hkv8TbVKaRo4mvPLb6RuaG6cgPtAeY3x2ZuW9,3.05000,2021-09-25T18:32:24.006Z,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,Top Crowdloaner
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,5FA7j7DgkdfzeaV4Xx41unUNftW9FXbXKYTRJU8FBYhQ8hPf,3.05000,2021-09-25T10:54:48.008Z,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,Top Crowdloaner
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,5GBdQv7trYp148eeiJ53Piwnjx5dSGn2GLeScQYTMf7aCyok,3.04570,2021-09-24T19:57:06.012Z,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,Top Crowdloaner
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,5CQJbQAdf5Ri2dRQ2Gqgz4DMV4PNmLoBjGJHTvu7nVvy96sB,3.04000,2021-09-24T22:29:36.014Z,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,Top Crowdloaner


What do people do with their KBTC?

In [32]:
my_currencies = ['kint', 'ksm', 'kbtc']

# def add_more_labels(my_currency):
agg = {}
for c in my_currencies:
  agg[c] = sum

top_transfer = xtoken_transfers.groupby(['from_id', 'fromChain','to_id', 'toChain']).agg(agg)
top_transfer.reset_index(inplace=True)


for my_currency in my_currencies:
  q = top_transfer[my_currency].quantile(.8)

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].from_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Mover')

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].to_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Sink')

enrich_df(top_transfer, 'from_id')
enrich_df(top_transfer, 'to_id')

top_transfer.sort_values('kbtc', ascending=False)


,from_id,fromChain,to_id,toChain,kint,ksm,kbtc,label_from_id,label_to_id
1715,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2000,0.00000,0.0,4.810259,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
1711,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0x63e33447838b9025e5cfdb35d4acc12a38342614,2023,0.00000,0.0,2.840376,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,Top KBTC Sink
1698,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,2092,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,2092,11.00000,0.0,1.735892,Top Issuer/Daddy of TzRzQ/Likely Vault Owner/T...,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
691,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0.50000,0.0,1.586544,Top Issuer/Top KBTC Mover/Top KBTC Sink,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
1751,a3dV3XyhVP3f1B1dwCvW7V8k87M2maqRxLpk8bb5f4TbPVSFD,2092,a3dV3XyhVP3f1B1dwCvW7V8k87M2maqRxLpk8bb5f4TbPVSFD,2000,0.00000,0.0,1.476546,Top Issuer/Top KINT Mover/Top KINT Sink/Top KB...,Top Issuer/Top KINT Mover/Top KINT Sink/Top KB...
...,...,...,...,...,...,...,...,...,...
931,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,a3ajGTepDb8rKFEugQHAgvmbhLDPiyxuKWNimtgBPU9Hfa8gy,2092,80.96325,0.0,0.000000,shenanigans/Top KINT Mover/Top KINT Sink,Top KINT Sink
932,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,a3aksvKs6KcGjiwEvRbJbwmJFhA2CfGq85Guk7ZEQ8oijMzpg,2092,5.10610,0.0,0.000000,shenanigans/Top KINT Mover/Top KINT Sink,
933,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,a3amD4HYMXaRhjyaC7j8V3ECf8jbpExLgbsJ7Weo7vtt7nms8,2092,0.79107,0.0,0.000000,shenanigans/Top KINT Mover/Top KINT Sink,
934,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,a3amZCwhEBoyAbhHMM58ZbizsyiPymYjoFjWyFVkjCCaAtmje,2092,3.06108,0.0,0.000000,shenanigans/Top KINT Mover/Top KINT Sink,


Let's try to see which accounts are likely CEX accounts, i.e. they receive large sums from many addresses. So we aggregate by number of distinct "from IDs" and sum up KINT. To understand "history", we also look at the first and last transfer for each account.

In [33]:
likely_kraken = "a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p"
xtoken_transfers['first'] = xtoken_transfers.timestamp # duplicate to determine first transfer
xtoken_transfers.groupby('to_id').agg({
    "from_id" : "nunique",
    "first" : min,
    "timestamp" : max,
    "kint" : sum,
    "kbtc" : sum,
}).sort_values('from_id', ascending=False)

,from_id,first,timestamp,kint,kbtc
to_id,,,,,
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,132,2022-02-21T13:02:42.294000Z,2022-06-03T00:44:30.542000Z,80505.557305,0.0
a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,88,2022-02-20T14:13:12.300000Z,2022-06-16T22:12:54.463000Z,101121.984567,0.0
a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,22,2022-06-05T03:50:24.392000Z,2022-06-16T12:21:00.458000Z,11608.237841,0.0
a3awFkWwydtvXZaQdQqpTeRAQpehjEe6wh57i6NTnka8xxKkw,20,2022-04-08T15:41:36.378000Z,2022-04-08T16:05:30.257000Z,139.997250,0.0
a3d28NjUAJzQdBSdv7W2CgNKD3wY7GPcYfcDyffCnGDAiz2Ef,19,2022-03-05T16:36:42.266000Z,2022-03-05T16:50:42.251000Z,91.180000,0.0
...,...,...,...,...,...
a3c5aFJoQNbhB1T6C9ZLDpD6NXXU1kdQfNA7gLjohSU5VEE9Q,1,2022-02-22T23:06:00.222000Z,2022-02-22T23:21:36.299000Z,4.670000,0.0
a3c5M82ZGZtTb5JJwQjDeehErXBu2YfJAscvW6bMx4PkWsLmh,1,2022-05-10T08:26:12.357000Z,2022-05-10T08:29:24.467000Z,1.560000,0.0
a3c5GAnjaWFjhiMGvnK8X1ZoisE9u1QNqsDktQ9vKGPNnJ8mP,1,2022-03-05T16:57:42.348000Z,2022-03-05T17:00:36.410000Z,97.790000,0.0


With some additional criteria based on the shape of the data (from ID>10, KINT>140) 
It seems that the the top 4 accounts

* a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW
* a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p
* a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC
* a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT

 are CEX accounts, with TdFW replaced by FiaXC (look at timestamps). Maybe the exchange needed to change the wallet address for some reason. It seems plausible because there was a gap of about 48 hours. 

Same story, but with KINT - labelling two accounts which seem to be system accounts

In [34]:
agg['to_id']='nunique'
agg['timestamp']='min'
xtoken_transfers['last_seen'] = xtoken_transfers.timestamp # workaround to agg this col, too
agg['last_seen']='max'

top_transfer_from = xtoken_transfers.groupby(['from_id', 'fromChain','toChain']).agg(agg)
top_transfer_from.kint = top_transfer_from.kint.apply(lambda k: round(k))
top_transfer_from = top_transfer_from.reset_index(inplace=False)

exclude = top_transfer_from.sort_values('kint', ascending=False).iloc[0:2, 0]
list(exclude)
add_label_to_list(exclude, 'KINT System?')
enrich_df(top_transfer_from, 'from_id')

In [35]:
top_transfer_from.sort_values('kint', ascending=False)

,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id
455,a3cgeH7D28bBsHWJtUcHf7srz25o34gCKi8SZVjky6nMyEm83,2092,2092,1325000,0.0,0.000000,2,2022-03-16T20:11:24.342000Z,2022-03-24T20:29:54.518000Z,Top KINT Mover/KINT System?
49,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,2092,2092,153801,0.0,0.000000,554,2022-02-21T13:48:30.573000Z,2022-06-17T07:25:30.312000Z,shenanigans/Top KINT Mover/Top KINT Sink/KINT ...
299,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,59830,0.0,0.000000,171,2022-02-20T00:37:12.309000Z,2022-06-16T17:42:54.286000Z,shenanigans/Top KINT Mover/Top KINT Sink
694,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,2092,2092,49655,0.0,0.000000,3,2022-03-29T14:37:30.396000Z,2022-06-05T14:54:18.296000Z,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
415,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,46886,0.0,0.000000,82,2022-02-17T17:37:06.708000Z,2022-06-16T09:21:06.310000Z,shenanigans/Top KINT Mover/Top KINT Sink
...,...,...,...,...,...,...,...,...,...,...
571,a3dJmVwmyQy9Wscq9Vokz9yw7coy6epe3nW94R6x1xwCrCXSH,2092,2000,0,0.0,0.008065,1,2022-05-08T01:23:06.486000Z,2022-05-08T01:23:06.486000Z,Top KBTC Mover/Top KBTC Sink
569,a3dJcbrvGrFBfPdN6DBjwgRGbMDdQCr2YFAgNkPzRJA2aRSPi,2092,2092,0,0.0,0.000000,1,2022-03-26T10:39:54.452000Z,2022-03-26T10:39:54.452000Z,
559,a3dFJpAppsqitaayxm4RevW1zRSh4PGYA3LzCX88vmS8WJEk3,2092,2092,0,0.0,0.000000,1,2022-03-28T15:49:54.759000Z,2022-03-28T15:49:54.759000Z,
556,a3dEwdYZQkL8xd8NTousR7ZB7VyWvthb9Fh4duiXjJDorQtvn,2092,2000,0,0.0,0.092974,1,2022-05-18T19:25:48.677000Z,2022-05-23T21:17:06.428000Z,Top Issuer/Top KBTC Mover/Top KBTC Sink


Let's bring in a temporal dimension and explore when accounts became active and when they last did something.

In [36]:
def calc_secs(row):
  return (pd.Timestamp(row['last_seen'])-pd.Timestamp(row['timestamp'])) // pd.Timedelta("1s")

In [37]:
enrich_df(top_transfer_from, 'from_id')
top_transfer_from['duration'] = top_transfer_from.apply(calc_secs, axis=1)
# top_transfer_from.loc[top_transfer_from.kbtc>0.025].sort_values('duration', ascending=False)
top_transfer_from.sort_values('duration', ascending=False).reset_index()
# top_transfer_from.sort_values('timestamp', ascending=True)

,index,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id,duration
0,415,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,46886,0.0,0.0,82,2022-02-17T17:37:06.708000Z,2022-06-16T09:21:06.310000Z,shenanigans/Top KINT Mover/Top KINT Sink,10251839
1,299,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,59830,0.0,0.0,171,2022-02-20T00:37:12.309000Z,2022-06-16T17:42:54.286000Z,shenanigans/Top KINT Mover/Top KINT Sink,10083941
2,1040,a3fqG9iWWv15rxGRviVutvjLEy9Dk42Gm9mNoNzPVbUKtfKWP,2092,2092,828,0.0,0.0,2,2022-02-20T21:13:30.212000Z,2022-06-16T17:00:36.291000Z,Top Crowdloaner/Top KINT Mover,10007226
3,583,a3dMNT1vy3fvLkBydZADzNQ3WPz449h3Fgoo3VSQ6k3Kqfchm,2092,2092,858,0.0,0.0,2,2022-02-20T21:14:24.310000Z,2022-06-16T17:00:06.739000Z,Top Crowdloaner/Top KINT Mover,10007142
4,49,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,2092,2092,153801,0.0,0.0,554,2022-02-21T13:48:30.573000Z,2022-06-17T07:25:30.312000Z,shenanigans/Top KINT Mover/Top KINT Sink/KINT ...,9999419
...,...,...,...,...,...,...,...,...,...,...,...,...
1071,443,a3cdPu3pFVPkwH1PYicAWKPKK4spLsecdGAXmDNzJjP7Z9fxm,2092,2092,9,0.0,0.0,1,2022-04-07T07:37:12.212000Z,2022-04-07T07:37:12.212000Z,,0
1072,444,a3cevVHdVrubHuuc17dzNah9EJSrT7zwLF5JTfdt1XBmzUsVr,2092,2092,4,0.0,0.0,1,2022-04-27T00:50:30.500000Z,2022-04-27T00:50:30.500000Z,Top Crowdloaner,0
1073,445,a3cewdxZHhq7ayJcuPUxABgUfUbr3Fga5gxNi9kdBrs6HjACF,2092,2092,1,0.0,0.0,1,2022-03-18T10:19:36.348000Z,2022-03-18T10:19:36.348000Z,,0
1074,446,a3ceycSRoqkqBmB5uEfRhhJhd89HVQcN2U3MPYzWbCdtEpAtb,2092,2092,33,0.0,0.0,1,2022-04-14T12:59:42.867000Z,2022-04-14T12:59:42.867000Z,,0


In [38]:
kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain",
)
#df = pd.DataFrame(kint_substrate.get_metadata_storage_functions())
p = {'account_id': 'a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS',
 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}}
result = kint_substrate.query(
    module='VaultRegistry',
    storage_function='Vaults',
    params=[p]
)
result.serialize()

{'active_replace_collateral': 0,
 'banned_until': None,
 'id': {'account_id': 'a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS',
  'currencies': {'collateral': {'Token': 'KSM'},
   'wrapped': {'Token': 'KBTC'}}},
 'issued_tokens': 78059731,
 'liquidated_collateral': 0,
 'replace_collateral': 0,
 'status': {'Active': False},
 'to_be_issued_tokens': 0,
 'to_be_redeemed_tokens': 0,
 'to_be_replaced_tokens': 0,
 'wallet': {'addresses': [{'P2WPKHv0': '0x0825c01d4ccd3a8d8746064e886ed662ba70e391'},
   {'P2WPKHv0': '0x0c12c9299e991b9ff993015a8e40c329ff50a6dd'},
   {'P2WPKHv0': '0x11f8e31b5644a052dd2f4dbedad5cdbb829818a3'},
   {'P2WPKHv0': '0x142929f7e233f1e1e4ac21d9312d5d58ef7d67c3'},
   {'P2WPKHv0': '0x1dade113727d63f4b168eb3fa3042aa6df5fa189'},
   {'P2WPKHv0': '0x1e3581a93453660adb5d24056eba778945142607'},
   {'P2WPKHv0': '0x1ea85f05614d6855161d036ae2bd0f5d865c6b70'},
   {'P2WPKHv0': '0x1fafdceb55c3c1882c8cdd48290fb214d43db131'},
   {'P2WPKHv0': '0x279442c5538c896226dd8a86c4ac7d6e6e8d0ffd'

This is how I figure out how to use the VaultRegistry / Vaults storage method. I call "query_map" and inspect the first output. The first part of the output
is the ID (param) to call for the storage function.
The second part is the output.

In [39]:
kint_substrate = SubstrateInterface(url="wss://api-kusama.interlay.io/parachain")

acct = list(kint_substrate.query_map("VaultRegistry", "Vaults"))
acct[0]


[<scale_info::68(value={'account_id': 'a3d2HzjdJoePv7ttTkGJXS6X65k1aaG1Q9HXQrRU2ThLAZKZZ', 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}})>,
 <scale_info::361(value={'id': {'account_id': 'a3d2HzjdJoePv7ttTkGJXS6X65k1aaG1Q9HXQrRU2ThLAZKZZ', 'currencies': {'collateral': {'Token': 'KSM'}, 'wrapped': {'Token': 'KBTC'}}}, 'wallet': {'addresses': [{'P2WPKHv0': '0x9a489168836ccdcc1e86a02779807daa927ff7a6'}]}, 'status': 'CommittedTheft', 'banned_until': None, 'to_be_issued_tokens': 0, 'issued_tokens': 0, 'to_be_redeemed_tokens': 0, 'to_be_replaced_tokens': 0, 'replace_collateral': 0, 'active_replace_collateral': 0, 'liquidated_collateral': 0})>]

In [40]:
def get_collateral_ksm(vault_id, kint_substrate):
  #with SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com") as kint_substrate:
  result = kint_substrate.query(
      module='Tokens',
      storage_function='Accounts',
      params=[vault_id,
              {'Token': 'KSM'}]
  )
  c = result.serialize().get('reserved') / 1e12
  return c

#kint_substrate = SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com")
#get_collateral_ksm('a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS', kint_substrate)

In [41]:
with SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com") as kint_substrate:
  result = kint_substrate.query_map("VaultRegistry", "Vaults")
new_list = [r[1].serialize() for r in result]


In [42]:
v = pd.json_normalize(new_list)
v.rename(columns={"id.account_id":"vault", "status.Active":"active"}, inplace=True)

master_t = dict(
    KSM = 1e12,
    BTC = 1e8,
    KBTC = 1e8,
    KINT = 1e12,
)

v['issued_kbtc'] = v.issued_tokens.apply(lambda x:round( x/1e8, 6))
v['liquidated_ksm'] = v.liquidated_collateral.apply(lambda x: round(x/1e12))
 
master_2 = v.sort_values('issued_kbtc', ascending=False).loc[v.issued_kbtc>.1, ['vault', 'status', 'active', 'issued_kbtc', 'liquidated_ksm']]
kint_substrate = SubstrateInterface(url="wss://kintsugi-rpc.dwellir.com")

master_2['collateral_ksm'] = master_2.vault.apply(get_collateral_ksm, kint_substrate=kint_substrate)
master_2['collateral_ksm'] = master_2['collateral_ksm'].apply(round)

In [43]:
enrich_df(master_2, 'vault')
master_2

,vault,status,active,issued_kbtc,liquidated_ksm,collateral_ksm,label_vault
21,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,NaN,True,10.219012,0,10994,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
13,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,NaN,False,4.099362,0,4167,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
37,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,NaN,True,0.785884,0,798,@warinelly/Social/Top 20 Vault/uUvS1/K>50/Top ...
47,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,NaN,False,0.780597,0,837,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,NaN,False,0.667683,0,1275,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
58,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,NaN,False,0.630625,0,898,@paride/Social/Top 20 Vault/K>50/Top KINT Move...
19,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,NaN,False,0.569978,0,579,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
14,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,NaN,True,0.540269,0,547,Social/Top 20 Vault/K>50/Top KINT Mover/Top KI...
26,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,NaN,False,0.513667,0,604,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
39,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,NaN,True,0.505425,0,517,Social/Top 20 Vault/H8Qem/K>50/Top KINT Mover/...


In [44]:
kint_substrate = SubstrateInterface(url="wss://api-kusama.interlay.io/parachain")

oracle = list(kint_substrate.query_map("Oracle", "Aggregate"))
ksm_per_btc = oracle[1][1].serialize()/1e22
ksm_per_btc

422.2527182518737

In [45]:
def get_account_balances(id, token):
  result = kint_substrate.query(
      module='Tokens',
      storage_function='Accounts',
      params=[id,
              {'Token': token}]
  )
  c = result.serialize().get('free') / 1e8
  return c

master_2['kbtc_balance'] = master_2.vault.apply(get_account_balances, token='KBTC')


In [46]:
master_2

,vault,status,active,issued_kbtc,liquidated_ksm,collateral_ksm,label_vault,kbtc_balance
21,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,NaN,True,10.219012,0,10994,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...,0.027352
13,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,NaN,False,4.099362,0,4167,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...,3.994770
37,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,NaN,True,0.785884,0,798,@warinelly/Social/Top 20 Vault/uUvS1/K>50/Top ...,0.000000
47,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,NaN,False,0.780597,0,837,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...,0.076955
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,NaN,False,0.667683,0,1275,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,0.005738
58,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,NaN,False,0.630625,0,898,@paride/Social/Top 20 Vault/K>50/Top KINT Move...,0.001243
19,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,NaN,False,0.569978,0,579,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...,1.117808
14,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,NaN,True,0.540269,0,547,Social/Top 20 Vault/K>50/Top KINT Mover/Top KI...,0.000381
26,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,NaN,False,0.513667,0,604,pumpernickel/Top Redeemer/Social/Top 20 Vault/...,0.000418
39,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,NaN,True,0.505425,0,517,Social/Top 20 Vault/H8Qem/K>50/Top KINT Mover/...,0.000000


In [47]:
kint_substrate = SubstrateInterface(url="wss://api-kusama.interlay.io/parachain")



In [48]:
#kint_whales = pd.Series(data=[k for k in kts if not "0x" in k])
#kint_whales = pd.DataFrame(data=kint_whales.apply(get_account_balances, token='KINT'), index=list(kint_whales))
#kint_whales.iloc[:, 0].sort_values(ascending=False)
#kint_whales.columns

In [49]:
f = funding_accounts.groupby(['daddy','label_daddy']).size().reset_index()
f['kbtc_balance'] = f.daddy.apply(get_account_balances, token='KBTC')
f['kint_balance'] = f.daddy.apply(get_account_balances, token='KINT')
f['kint_balance'] = f['kint_balance'].apply(lambda x: round(x/1e4))
f.sort_values('kint_balance', ascending=False)


,daddy,label_daddy,0,kbtc_balance,kint_balance
19,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,Daddy of ceZ9g/Likely Vault Owner,1,0.000000,73580
12,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,2,0.005738,10851
18,a3c7uUVvpdHUySP3oJ3W16uQPN9cyAa7AxLmMmGsQAQmrEszH,Social/Top 20 Vault/Daddy of Poxg8/rEszH/Likel...,1,0.000529,1106
32,a3eHgybGLoM4FpFPcWpyUdGVwrMYVUNcGvmZbRfhZZwj1pzms,Top Issuer/Daddy of ZhL2w/Likely Vault Owner,1,0.000010,232
27,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,1,0.099250,77
6,a3awoQLGoQNGvR7UuXvgA7WhEk954SYhLuxvXASenQ8z1LUQH,Top Issuer/Daddy of Qupbw/Likely Vault Owner,1,0.000000,61
36,a3ev35BqB72koybME5Zkmho9E8oHABWx9RKeGVzvYx8aM1cho,Daddy of KueH6/Likely Vault Owner,1,0.000000,41
26,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,Top Issuer/Daddy of TzRzQ/Likely Vault Owner,1,0.000000,38
0,a3aDFMF8Nq4cEcE29gtxJtoUnLSVa9JbozNBYefn2T9ioThRr,Daddy of H8Qem/Likely Vault Owner,1,0.000000,38
30,a3dkY3yWweLMkUjbSSNC2NawWegH3NZkWpYjsnBr2srQsp9LZ,Daddy of 3yHRD/Likely Vault Owner,1,0.000000,35


In [50]:
master_2['coll_ratio'] = 100 * (master_2['collateral_ksm'] / ksm_per_btc ) / master_2['issued_kbtc']
master_2['coll_ratio'] = master_2['coll_ratio'].apply(round)
master_2.sort_values('coll_ratio', ascending=False).reset_index().loc[:,set(master_2.columns)-{"index", "status", "liquidated_ksm"}]

,active,collateral_ksm,vault,issued_kbtc,coll_ratio,label_vault,kbtc_balance
0,False,1275,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,0.667683,452,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...,0.005738
1,False,714,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,0.449225,376,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,0.105424
2,False,898,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.630625,337,@paride/Social/Top 20 Vault/K>50/Top KINT Move...,0.001243
3,False,270,a3fRJZLKJEsNY2fcDDLtzh7MWAE5DbVykyEfBbKdPbJSGLQjh,0.201952,317,Social/Vault/Self Issuer/GLQjh/Top KINT Mover/...,0.000379
4,False,138,a3cS7bP56bj11Yrfxt3TZFGjo96R7eJH6WUNYBxg1dx55jCJm,0.115836,282,Selfish/Vault/Self Issuer/Top KINT Mover,0.095098
5,False,526,a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed,0.443178,281,@spazcoin/chaos DAO/Top Issuer/Selfish/Top 20 ...,0.327427
6,False,604,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,0.513667,278,pumpernickel/Top Redeemer/Social/Top 20 Vault/...,0.000418
7,False,189,a3fudELrRCjuSyYEPkRAKFQyjzo5YyU228LdqinGsnjBUNB8P,0.162036,276,@spazcoin/chaos DAO/VaaS/Top Issuer/Selfish/Va...,0.150644
8,False,207,a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4,0.188393,260,@timbotronic/Top Issuer/Selfish/Vault/Self Iss...,0.000000
9,False,199,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,0.183296,257,@seergeist/Top Redeemer/Top Issuer/Selfish/Vau...,0.018908


# Next few cells are for ad-hoc analysis

In [51]:
xtoken_transfers.columns

Index(['amount', 'dot', 'from.karura', 'from.kintsugi', 'from.kusama',
       'from.moonriver', 'fromChain', 'from_id', 'kbtc', 'kint', 'ksm',
       'timestamp', 'to.karura', 'to.kintsugi', 'to.kusama', 'to.moonriver',
       'toChain', 'to_id', 'token', 'first', 'last_seen'],
      dtype='object')

In [52]:
from datetime import datetime, timedelta, timezone

now = datetime.now(timezone.utc)
earlier = pd.to_datetime(now - timedelta(days=7))
earlier

Timestamp('2022-06-10 09:14:56.912021+0000', tz='UTC')

In [53]:
v="a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe"
xtoken_transfers['timestamp_dt'] = xtoken_transfers.timestamp.apply(pd.to_datetime)
xtoken_transfers['ago'] = xtoken_transfers.timestamp_dt.apply(lambda t: now-t)
enrich_df(xtoken_transfers, 'to_id')
xtoken_transfers.loc[(xtoken_transfers['from_id']==v) & 
                     (xtoken_transfers.token=='KINT') &
                     (xtoken_transfers.timestamp_dt > earlier), ['ago','kint', 'kbtc', 'timestamp', 'to_id', 'toChain']]


,ago,kint,kbtc,timestamp,to_id,toChain
3543,0 days 14:51:32.746021,125.0,NaN,2022-06-16T18:23:24.166000Z,a3fxEoowRoArhDxv9TdcJS7hXGdG6vgw3XhzPkGLVqfwgXjHK,2092
14943,2 days 14:21:26.447021,70.0,NaN,2022-06-14T18:53:30.465000Z,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,2000
20843,6 days 19:05:20.518021,150.0,NaN,2022-06-10T14:09:36.394000Z,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,2000


In [54]:
_.kint.sum()

345.0

In [55]:
kts['a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3']

['Top Redeemer',
 'Top Issuer',
 'Selfish',
 'Top 20 Vault',
 'Self Issuer',
 'Daddy of BdzVR',
 'Daddy of g5rez',
 'LLDD3',
 'Daddy of 9v6eN',
 'Likely Vault Owner',
 'K>50',
 'Top KINT Mover',
 'Top KINT Sink',
 'Top KBTC Sink']

In [56]:
kts['a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR']

['Vault', 'Self Issuer', 'BdzVR', 'K>50', 'Top KINT Mover', 'Top KSM Sink']

In [57]:
premiums = """
query MyQuery {
  redeemExecutions(orderBy: timestamp_DESC, where: {redeem: {collateralPremium_gt: "0"}}) {
    timestamp
    redeem {
      status
      userBackingAddress
      userParachainAddress
      vault {
        accountId
      }
      collateralPremium
      request {
        requestedAmountBacking
      }
    }
  }
}

"""

In [58]:
r = requests.post(kintsugi, json={"query" : premiums}).json()
premium_redeems = pd.json_normalize(r['data']['redeemExecutions'])
premium_redeems['ksm_premium'] = premium_redeems['redeem.collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems['btc_requested'] = premium_redeems['redeem.request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)

premium_redeems.loc[premium_redeems.timestamp>"2022-06-10"].groupby('redeem.userParachainAddress').agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
        "timestamp": max,
     
    }
).reset_index().rename(columns={"redeem.userParachainAddress":"user"}, inplace=False).sort_values('ksm_premium', ascending=False)


,user,ksm_premium,btc_requested,timestamp
5,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,36.050921,1.439581,2022-06-12T04:37:12.502Z
6,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,7.449030,0.298392,2022-06-12T04:40:06.463Z
4,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,5.452749,0.217518,2022-06-12T04:37:12.502Z
0,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z
1,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,3.733509,0.149104,2022-06-12T04:37:12.502Z
7,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,1.769771,0.065336,2022-06-14T05:03:18.358Z
2,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,0.275274,0.010849,2022-06-13T02:32:06.252Z
3,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.219618,0.007923,2022-06-14T05:03:18.358Z


In [59]:
redeem_cxl = """
query MyQuery {
  redeemCancellations(orderBy: timestamp_DESC, where: {redeem: {collateralPremium_gt: "75639044748"}}) {
    timestamp
    redeem {
      status
      userBackingAddress
      userParachainAddress
      vault {
        accountId
      }
      collateralPremium
      request {
        requestedAmountBacking
      }
    }
  }
}
"""

In [60]:
r = requests.post(kintsugi, json={"query" : premiums}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeemExecutions'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['redeem.collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['redeem.request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby('redeem.userParachainAddress').agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
        "timestamp": max,
     
    }
).reset_index().rename(columns={"redeem.userParachainAddress":"user"}, inplace=False).sort_values('ksm_premium', ascending=False)


,user,ksm_premium,btc_requested,timestamp
16,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,45.080122,2.273760,2022-05-14T20:19:30.305Z
11,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,40.570900,1.687699,2022-06-12T04:37:12.502Z
14,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,14.024796,0.617199,2022-06-12T04:40:06.463Z
9,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,11.846199,0.565476,2022-06-12T04:37:12.502Z
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,10.225996,0.524853,2022-05-15T10:13:18.321Z
19,a3fTkM2XKvxD4rGA4UHt8Efr9MgvLZH6mvYGtKdMctaLr4fdV,9.340200,0.634175,2022-05-03T00:10:24.620Z
4,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z
5,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,4.055474,0.165112,2022-06-12T04:37:12.502Z
20,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,2.082392,0.079109,2022-06-14T05:03:18.358Z
21,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.337879,0.068667,2022-05-15T10:13:18.321Z


In [61]:
df = _
enrich_df(df, 'user')
df.ksm_premium.sum() / df.btc_requested.sum() / 400

0.05229388166532546

In [62]:
df

,user,ksm_premium,btc_requested,timestamp,label_user
16,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,45.080122,2.273760,2022-05-14T20:19:30.305Z,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
11,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,40.570900,1.687699,2022-06-12T04:37:12.502Z,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...
14,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,14.024796,0.617199,2022-06-12T04:40:06.463Z,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...
9,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,11.846199,0.565476,2022-06-12T04:37:12.502Z,Top Redeemer/Top Issuer/Social/Vault/Daddy of ...
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,10.225996,0.524853,2022-05-15T10:13:18.321Z,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
19,a3fTkM2XKvxD4rGA4UHt8Efr9MgvLZH6mvYGtKdMctaLr4fdV,9.340200,0.634175,2022-05-03T00:10:24.620Z,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...
4,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,4.976767,0.198908,2022-06-12T04:37:12.502Z,Top Issuer/Top KBTC Mover/Top KBTC Sink
5,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,4.055474,0.165112,2022-06-12T04:37:12.502Z,pumpernickel/Top Redeemer/Social/Top 20 Vault/...
20,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,2.082392,0.079109,2022-06-14T05:03:18.358Z,Daddy of Qupbw/Likely Vault Owner/Top KINT Mov...
21,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.337879,0.068667,2022-05-15T10:13:18.321Z,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...


In [63]:
# retries only
retries = """
query MyQuery {
  redeems(where: {cancellation: {slashedCollateral_gt: "0"}}, orderBy: collateralPremium_DESC) {
    status
    vault {
      accountId
      registrationTimestamp
    }
    collateralPremium
    request {
      requestedAmountBacking
      timestamp
    }
    userParachainAddress
    cancellation {
      reimbursed
      timestamp
      slashedCollateral
    }
  }
}

"""

In [64]:
r = requests.post(kintsugi, json={"query" : retries}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeems'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['ksm_slashed'] = premium_redeems_x['cancellation.slashedCollateral'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby(['vault.accountId','userParachainAddress', 'status', ]).agg(
    {
        "ksm_premium": sum,
        "ksm_slashed": sum,
        "btc_requested": sum,
        "cancellation.timestamp": max,
     
    }
).reset_index().rename(columns={"userParachainAddress":"user",
                                "cancellation.timestamp": "last_action"
                            }, inplace=False).sort_values('ksm_slashed', ascending=False)


,vault.accountId,user,status,ksm_premium,ksm_slashed,btc_requested,last_action
4,a3b3EwCtmURY7K3d6aoWzouHriGfTsvCP2axuMVGpRpkPoxg8,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Reimbursed,0.000000,20.083839,0.063478,2022-05-06T06:19:54.316Z
9,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Retried,9.556306,19.525180,0.490481,2022-05-14T05:45:42.273Z
7,a3chQwNk9sxP4WHnm82cD6a5AR2tXJ7bmJ88yZGQp4Ay957PB,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Retried,3.737476,7.915483,0.198931,2022-05-14T05:23:42.583Z
8,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,a3bAsW2TQb1c7HrcZd2y81SdCVSGEA5pRkfg576G2EyxQupbw,Reimbursed,0.308839,7.005320,0.015851,2022-05-14T06:25:48.346Z
0,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4,Reimbursed,0.000000,2.280142,0.004902,2022-05-14T12:51:18.485Z
10,a3e2AF9D4WUhuZPZvNiAkW2eWGLfMWQfBYWzwgk8bhmHF3LDc,a3e22ukKfJ4ZPNzXbpfrcndUgPHbxw4BUox4t4J7BeK6tJe7q,Reimbursed,0.000000,1.922981,0.004804,2022-05-25T11:09:06.352Z
5,a3bBUocXg2cDMctxmEUaQPteH67NyLqxrYwX7soHuZaCHV5nE,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Retried,0.377534,1.794264,0.042163,2022-06-10T06:47:42.342Z
6,a3bBUocXg2cDMctxmEUaQPteH67NyLqxrYwX7soHuZaCHV5nE,a3crPwJ717zKVvpP8MWscjfVoXgwETmLhpAppat6P5fPLqKSb,Reimbursed,0.000000,1.374952,0.004929,2022-04-06T15:32:24.269Z
1,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Retried,0.000000,0.902337,0.029994,2022-05-08T08:52:48.305Z
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Retried,0.000000,0.804636,0.019833,2022-05-14T09:36:00.368Z


In [65]:
df = _
enrich_df(df, 'user')
df.ksm_slashed.sum() / df.btc_requested.sum() / 400

0.1815387491577276

In [66]:
q = """
query MyQuery {
  redeems(orderBy: request_timestamp_DESC, where: {collateralPremium_gt: "0"}) {
    request {
      requestedAmountBacking
      timestamp
    }
    status
    userParachainAddress
    vault {
      accountId
    }
    collateralPremium
  }
}
"""



In [67]:
r = requests.post(kintsugi, json={"query" : q}).json()
premium_redeems_x = pd.json_normalize(r['data']['redeems'])
premium_redeems_x['ksm_premium'] = premium_redeems_x['collateralPremium'].apply(lambda x: float(x)/1e12)
premium_redeems_x['btc_requested'] = premium_redeems_x['request.requestedAmountBacking'].apply(lambda x: float(x)/1e8)
premium_redeems_x.groupby(['userParachainAddress', 'status', 'vault.accountId']).agg(
    {
        "ksm_premium": sum,
        "btc_requested": sum,
     "request.timestamp": max,
     
    }
).reset_index().rename(columns={"userParachainAddress":"user",
                                "request.timestamp": "last_action"
                            }, inplace=False).sort_values('last_action', ascending=False)


,user,status,vault.accountId,ksm_premium,btc_requested,last_action
13,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.813513,0.038405,2022-06-13T08:31:24.536Z
42,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,1.479197,0.053725,2022-06-13T07:23:24.258Z
10,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3b5jAhU7CKnBKA3y92HkiCqRTbSKRMK5kN3RVPXcKoH4SDaC,0.132526,0.004944,2022-06-13T01:09:54.238Z
5,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,4.976767,0.198908,2022-06-11T19:47:18.416Z
22,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,40.025560,1.652461,2022-06-11T18:40:54.604Z
17,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,6.876467,0.292502,2022-06-11T18:02:54.360Z
29,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,8.842710,0.372572,2022-06-11T17:28:30.592Z
7,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,3.733509,0.149104,2022-06-11T16:32:42.406Z
43,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.290574,0.011612,2022-06-11T14:06:54.297Z
11,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.142748,0.005905,2022-06-10T17:49:36.422Z


In [68]:
df = _
enrich_df(df, "user")
enrich_df(df, "vault.accountId")
df

,user,status,vault.accountId,ksm_premium,btc_requested,last_action,label_user,label_vault.accountId
13,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.813513,0.038405,2022-06-13T08:31:24.536Z,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...
42,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,1.479197,0.053725,2022-06-13T07:23:24.258Z,Daddy of Qupbw/Likely Vault Owner/Top KINT Mov...,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...
10,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3b5jAhU7CKnBKA3y92HkiCqRTbSKRMK5kN3RVPXcKoH4SDaC,0.132526,0.004944,2022-06-13T01:09:54.238Z,@seergeist/Top Redeemer/Top Issuer/Selfish/Vau...,Social/Vault/4SDaC/Top KINT Mover/Top KSM Sink
5,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,4.976767,0.198908,2022-06-11T19:47:18.416Z,Top Issuer/Top KBTC Mover/Top KBTC Sink,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
22,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,40.025560,1.652461,2022-06-11T18:40:54.604Z,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
17,a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,6.876467,0.292502,2022-06-11T18:02:54.360Z,Top Redeemer/Top Issuer/Social/Vault/Daddy of ...,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
29,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,8.842710,0.372572,2022-06-11T17:28:30.592Z,@alibaba/Top Redeemer/Top Issuer/Daddy of dXze...,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
7,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,3.733509,0.149104,2022-06-11T16:32:42.406Z,pumpernickel/Top Redeemer/Social/Top 20 Vault/...,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
43,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,Completed,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.290574,0.011612,2022-06-11T14:06:54.297Z,Daddy of Qupbw/Likely Vault Owner/Top KINT Mov...,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
11,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,Completed,a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs,0.142748,0.005905,2022-06-10T17:49:36.422Z,@seergeist/Top Redeemer/Top Issuer/Selfish/Vau...,@boyswan/Selfish/Vault/Top KINT Mover/Top KINT...


In [69]:
kts['a3chQwNk9sxP4WHnm82cD6a5AR2tXJ7bmJ88yZGQp4Ay957PB']

['Selfish',
 'Vault',
 'Self Issuer',
 '957PB',
 'K>50',
 'Top KINT Mover',
 'Top KSM Sink']